# Offline Step: Generation of Training Artifacts and Dataset Download

## Part 1: Training Artficat Generation

In [1]:
%pip install --upgrade pip
%pip install onnxruntime torch torchvision medmnist tqdm
%pip install cerberus flatbuffers h5py numpy onnx packaging protobuf sympy setuptools
%pip install -i https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ORT/pypi/simple/ onnxruntime-training-cpu
%pip install netron

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ORT/pypi/simple/
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# largely borrowed from https://www.youtube.com/watch?v=u7YCaiHOC9o

import torch
import os
import torchvision
import netron

os.makedirs("public", exist_ok=True)
os.makedirs("public/training_artifacts", exist_ok=True)

# load weights from pretrained resnet model
model = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.DEFAULT
)


# changing last layer to only output one of 8 classes since bloodMNIST only has 8
num_fc_feats = model.fc.in_features
model.fc = torch.nn.Linear(num_fc_feats, 8)

# convert the model to ONNX
model.train()
model_name = "resnet50"
torch.onnx.export(model,
                  torch.randn(1, 3, 224, 224),
                  f"public/training_artifacts/{model_name}.onnx",
                  input_names=["input"],
                  output_names=["output"],
                  dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
                  export_params=True,
                  do_constant_folding=False
)

netron.start(f"public/training_artifacts/{model_name}.onnx")

Serving 'public/training_artifacts/resnet50.onnx' at http://localhost:8080


('localhost', 8080)

Now we have a model converted to ONNX format, need to generate the training artifacts that we will use later:

In [3]:
import onnx
from onnxruntime.training import artifacts

# get the onnx model
onnx_model = onnx.load(f"public/training_artifacts/{model_name}.onnx")

# distinguish which weights we want to train and which we want to leave (in demo case, only last layer)
requires_grad = ["fc.weight", "fc.bias"]
frozen_params = [
    param.name
    for param in onnx_model.graph.initializer
    if param.name not in requires_grad
]

# generate the training artifacts
artifacts.generate_artifacts(
    onnx_model,
    requires_grad=requires_grad,
    frozen_params=frozen_params,
    loss=artifacts.LossType.CrossEntropyLoss,
    optimizer=artifacts.OptimType.AdamW,
    artifact_directory="public/training_artifacts"
)

Opening in existing browser session.


2024-11-24 08:12:22.716715816 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer ConstantSharing modified: 0 with status: OK
2024-11-24 08:12:22.717131410 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer LayerNormFusion modified: 0 with status: OK
2024-11-24 08:12:22.717234158 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer CommonSubexpressionElimination modified: 0 with status: OK
2024-11-24 08:12:22.717260398 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer GeluFusion modified: 0 with status: OK
2024-11-24 08:12:22.717287237 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer SimplifiedLayerNormFusion modified: 0 with status: OK
2024-11-24 08:12:22.717305907 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer FastGeluFusion modified: 0 with status: OK
2024-11-24 08:12:22.717321377 [I:onnxruntime:Default, graph_transformer.cc:15 Apply] GraphTransformer Qui

## Part 2: Dataset Download and Install

For the purposes of this demo, we install the bloodMNIST dataset and test using that.

In [ ]:
!python convert_dataset.py --data_flag "bloodmnist" --output_dir "public/data"

Using downloaded and verified file: /home/saafetensors/.medmnist/bloodmnist.npz
Using downloaded and verified file: /home/saafetensors/.medmnist/bloodmnist.npz
Processing train data...
Processing train samples:   0%|                       | 0/11959 [00:00<?, ?it/s]/home/saafetensors/Desktop/decai-fe/convert_dataset.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'label': int(label)
Processing train samples: 100%|████████| 11959/11959 [00:00<00:00, 27336.45it/s]
Saved train data to public/data/bloodmnist_train.json
Processing test data...
Processing test samples: 100%|███████████| 3421/3421 [00:00<00:00, 38447.37it/s]
Saved test data to public/data/bloodmnist_test.json
Conversion completed successfully.


: 